In [1]:
#importing libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd




In [2]:
#import data
dataset=pd.read_csv('50_Startups.csv')
x=dataset.iloc[:,:-1].values
y=dataset.iloc[:,4].values

dataset1=dataset.iloc[:,:-1]

In [3]:
dummy=pd.get_dummies(dataset['State'])
dataset1=pd.concat([dummy,dataset1],axis=1)
dataset1.drop('State',axis=1,inplace=True)

#dummy for purchase
x=dataset1.values

#avoiding dummy variable trap
#removes one dummy variable from the list
x=x[:,1:]

In [4]:
#backward elimination
import statsmodels.regression.linear_model as sm
#prep
x=np.append(arr=np.ones((50,1)).astype(int),values=x,axis=1)


#function for backward elimination
def backwardElimination(x, sl):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
    regressor_OLS.summary()
    return x
 
SL = 0.05
x_opt = x[:, [0, 1, 2, 3, 4, 5]]
x_Modeled = backwardElimination(x_opt, SL)

In [5]:
#splitting into test and train set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_Modeled,y,test_size=0.2,random_state=0)

In [6]:
#feature scaling not required for multiple regression
'''
from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
x_train=sc_x.fit_transform(x_train)
x_test=sc_x.transform(x_test)
sc_y=StandardScaler()
y_train=sc_x.fit_transform(y_train)
'''


'\nfrom sklearn.preprocessing import StandardScaler\nsc_x=StandardScaler()\nx_train=sc_x.fit_transform(x_train)\nx_test=sc_x.transform(x_test)\nsc_y=StandardScaler()\ny_train=sc_x.fit_transform(y_train)\n'

In [7]:
#fitting into model
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [8]:
#checking after backward elimination
regressor.fit(x_train,y_train)
new_pred=regressor.predict(x_test)

In [9]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
mse = mean_squared_error(y_test, new_pred)
print('Mean squared error:',mse)
r_score = r2_score(y_test, new_pred)
print('Accuracy:',r_score*100)
mae = mean_absolute_error(y_test, new_pred)
print('Mean squared error:',mae)

Mean squared error: 68473440.71905926
Accuracy: 94.6458760778722
Mean squared error: 6772.453280477901
